In [88]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

In [89]:
import os
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [90]:
# Load the dataset
df = pd.read_csv('../datasets/cleaned_twitter_dataset.csv')
df.columns = ['score', 'text',]

In [91]:
df.head()

,score,text
0,0,"['kenichan', 'dived', 'many', 'time', 'ball', ..."
1,0,"['whole', 'body', 'feel', 'itchy', 'like', 'fi..."
2,0,"['nationwideclass', 'behaving', 'im', 'mad', '..."
3,0,"['kwesidei', 'whole', 'crew']"
4,0,"['need', 'hug']"


In [92]:
df.shape

(1599998, 2)

In [93]:
# count missing values by colum
df.isnull().sum()

score    0
text     0
dtype: int64

In [94]:
# Remove rows with missing values
df = df.dropna(subset=['score', 'text'])
df.shape

(1599998, 2)

In [95]:
df.nunique()

score          2
text     1553368
dtype: int64

In [96]:
df['score'].nunique()

2

In [97]:
df['score'].unique()

array([0, 4])

In [98]:
df['score'] = df['score'].replace(4, 1)
df.tail()

,score,text
1599993,1,"['woke', 'school', 'best', 'feeling', 'ever']"
1599994,1,"['thewdbcom', 'cool', 'hear', 'old', 'walt', '..."
1599995,1,"['ready', 'mojo', 'makeover', 'ask', 'detail']"
1599996,1,"['happy', '38th', 'birthday', 'boo', 'alll', '..."
1599997,1,"['happy', 'charitytuesday', 'thenspcc', 'spark..."


In [99]:
df['score'].unique()

array([0, 1])

In [100]:
# Split the dataset into training and testing sets
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [101]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=200000, split=' ')
tokenizer.fit_on_texts(df_train['text'].values)

In [102]:
# Convert the text data to sequences and pad them
max_len = max([len(s.split()) for s in df_train['text'].values])
x_train = pad_sequences(tokenizer.texts_to_sequences(df_train['text'].values), maxlen=max_len)
x_test = pad_sequences(tokenizer.texts_to_sequences(df_test['text'].values), maxlen=max_len)

In [103]:
# Get the target variables
y_train = df_train['score']
y_test = df_test['score']

In [104]:
# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=200000, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [105]:
# Fit the model to the training data
os.environ['TF_METAL_DEVICE_PLACEMENT'] = 'metal:0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Reduce the number of samples used per epoch to 1000
batch_size = 64
steps_per_epoch = 1000
num_epochs = 10

model.fit(x_train, y_train, validation_split=0.1, batch_size=batch_size, epochs=num_epochs, steps_per_epoch=steps_per_epoch)

Epoch 1/10
1000/1000 [==============================] - 273s 272ms/step - loss: 0.5221 - accuracy: 0.7375 - val_loss: 0.4898 - val_accuracy: 0.7630
Epoch 2/10
1000/1000 [==============================] - 274s 274ms/step - loss: 0.4834 - accuracy: 0.7675 - val_loss: 0.4768 - val_accuracy: 0.7691
Epoch 3/10
1000/1000 [==============================] - 273s 273ms/step - loss: 0.4678 - accuracy: 0.7762 - val_loss: 0.4651 - val_accuracy: 0.7765
Epoch 4/10
1000/1000 [==============================] - 275s 275ms/step - loss: 0.4580 - accuracy: 0.7836 - val_loss: 0.4577 - val_accuracy: 0.7822
Epoch 5/10
1000/1000 [==============================] - 275s 275ms/step - loss: 0.4605 - accuracy: 0.7806 - val_loss: 0.4545 - val_accuracy: 0.7857
Epoch 6/10
1000/1000 [==============================] - 278s 278ms/step - loss: 0.4522 - accuracy: 0.7862 - val_loss: 0.4509 - val_accuracy: 0.7872
Epoch 7/10
1000/1000 [==============================] - 274s 274ms/step - loss: 0.4531 - accuracy: 0.7853 - val_

In [106]:
# Save model
model.save('final_models/lstm_e10_b64.h5')

In [107]:
# Evaluate model
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))
print('Loss: %f' % (loss*100))

Accuracy: 79.092187
Loss: 44.400445


In [108]:
# load the saved model
from keras.models import load_model
model = load_model('final_models/lstm_e10_b128_p20.h5')

# make predictions
y_pred = model.predict(x_test)

InvalidArgumentError: Graph execution error:

Detected at node 'sequential/embedding/embedding_lookup' defined at (most recent call last):
    File "/Users/thaveesha/Developer/miniconda3/envs/tensorflow/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/thaveesha/Developer/miniconda3/envs/tensorflow/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/Users/thaveesha/Developer/miniconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/thaveesha/Developer/miniconda3/envs/tensorflow/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/thaveesha/Developer/miniconda3/envs/tensorflow/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/thaveesha/Developer/m

In [ ]:
import matplotlib.pyplot as plt

plt.plot(y_test, label='actual')
plt.plot(y_pred, label='predicted')
plt.legend()
plt.show()

In [ ]:
# Make predictions
new_sentences = ['I am feeling sad today', 'Today is a great day']
new_sequences = tokenizer.texts_to_sequences(new_sentences)
new_sequences = pad_sequences(new_sequences)
predictions = model.predict(new_sequences)